In [1]:
import pandas as pd
import ast

In [2]:
main_data=pd.read_csv("real_domains.csv",names=["Top level domain","MX-Record from the name server","A-Record of the specific domain","Timestamp of scraping process"])#,
                      #converters={"Top level domain":str,"MX-Record from the name server":str,"Timestamp of scraping process":str})

In [3]:
main_data.head()

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process
0,0--1.de,"[""mail.0--1.de"",""mxf993.netcup.net""]","[""46.38.249.145""]",2020-12-13 15:36:05
1,0--2.de,"[""mxf993.netcup.net"",""mail.0--2.de""]","[""212.227.212.163""]",2020-12-13 15:36:05
2,0-0-0-0-0.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","[""80.150.6.143""]",2020-12-13 15:36:05
3,0-0-0-1.de,"[""smtp-02.tld.t-online.de"",""smtp-01.tld.t-onli...","[""80.150.6.143""]",2020-12-13 15:36:05
4,0-0.de,[],"[""185.53.178.13""]",2020-12-13 15:36:05


In [4]:
main_data[945622:945629]

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process
945622,derproviantmeister.de,"[""mx01.kundenserver.de"",""mx00.kundenserver.de""]","[""217.160.0.200""]",2020-12-09 20:04:42
945623,derprovider.de,"[""localhost""]","[""91.195.241.137""]",2020-12-09 20:04:42
945624,derproviderserver.de,"[""dps123.derproviderserver.de"",""dps13.derprovi...","[""83.243.40.214""]",2020-12-09 20:04:42
945625,derprovokateur.de,"[""localhost""]","[""91.195.241.137""]",2020-12-09 20:04:42
945626,derprozessor.de,"[""mx01.kundenserver.de"",""mx00.kundenserver.de""]","[""217.160.0.147""]",2020-12-09 20:04:42
945627,derpruefdienstleister.de,"[""mx00.ionos.de"",""mx01.ionos.de""]","[""217.160.0.61""]",2020-12-09 20:04:42
945628,derpsychologedesboesen.de,[],"[""94.130.50.195""]",2020-12-09 20:04:42


In [4]:
main_data[~(main_data["MX-Record from the name server"].str.contains(']',case=False))]

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process
945624,derproviderserver.de,"[""dps123.derproviderserver.de"",""dps13.derprovi...","[""83.243.40.214""]",2020-12-09 20:04:42
1048480,domservice.de,"[""mail01-f.domservice.de"",""mail01-c.domservice...","[""213.179.64.5""]",2020-12-09 22:26:48
3520446,redleadsmarketing.de,"[""99smtp-v0.redleadsmarketing.de"",""99smtp-v1.r...",[],2020-12-12 08:05:05


In [10]:
main_data[~(main_data["MX-Record from the name server"].str.contains(']',case=False))].index

Int64Index([945624, 1048480, 3520446], dtype='int64')

In [12]:
main_data_2 = main_data.drop([main_data.index[945624], main_data.index[1048480], main_data.index[3520446]])

In [47]:
main_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4860885 entries, 0 to 4860884
Data columns (total 4 columns):
 #   Column                           Dtype 
---  ------                           ----- 
 0   Top level domain                 object
 1   MX-Record from the name server   object
 2   A-Record of the specific domain  object
 3   Timestamp of scraping process    object
dtypes: object(4)
memory usage: 148.3+ MB


* Top level domain: Namensserver; das, was Personen sich merken können
* Ein MX Resource Record weist einem Namen einen Mailserver zu. Er stellt eine Besonderheit dar, da er sich auf einen speziellen Dienst im Internet, nämlich die E-Mailzustellung mittels SMTP, bezieht. Alle anderen Dienste nutzen CNAME, A und AAAA Resource Records für die Namensauflösung.
* Ein A Resource Record weist einem Namen eine IPv4-Adresse zu.
* alles Object Werte

In [48]:
main_data["Top level domain"].value_counts()

auge-herzberg.de                    1
xn--trockeneisstrahlen-sd-sic.de    1
pjog.de                             1
fomaonline.de                       1
elektro-kienhoefer.de               1
                                   ..
canelovsrockyfielding.de            1
kuehlen-umwelttechnik.de            1
oncoming.de                         1
pellets2go.de                       1
mytopaz.de                          1
Name: Top level domain, Length: 4860885, dtype: int64

In [49]:
tmp_vc=main_data["Top level domain"].value_counts()
tmp_vc[tmp_vc>2]

Series([], Name: Top level domain, dtype: int64)

* es gibt keine Domain, die doppelt vorkommt

In [69]:
#main_data.iloc[0,1].replace("\"\"", "\"")
main_data.iloc[0,1].list()

AttributeError: 'str' object has no attribute 'list'

In [13]:
main_data_2["MX-Record from the name server"]=main_data_2["MX-Record from the name server"].apply(lambda x: ast.literal_eval(x))

In [ ]:
main_data.head()

In [10]:
main_data.to_csv("test1.csv")

In [53]:
main_data["MX-Record from the name server"].value_counts()

[]                                             634415
[smtpin.rzone.de]                              509127
[localhost]                                    303839
[mx01.kundenserver.de,mx00.kundenserver.de]    176491
[mx00.kundenserver.de,mx01.kundenserver.de]    176182
                                                ...  
[mail.remotezuhause.de]                             1
[mail.rksoftware.de]                                1
[mail.dachbeschichtungsprofi.de]                    1
[mail.n8ly.de]                                      1
[w0129c7a.kasserver.com]                            1
Name: MX-Record from the name server, Length: 1294739, dtype: int64

* es gibt eine Kummulation bei bestimmten MX-Records. Das bedeutet, dass verschiedenste Domains den gleichen Mailserver nutzen (oder gar keinen).
* Teileweise gibt es mehrere MX-Records pro Domain

In [3]:
data={"os":[["smtp-02.tld.t-online.de","smtp-02.tld.t-online.de","smtp-01.tld.t-online.de"],"[smtp-01.tld.t-online.de]","[smtp-02.tld.t-online.de,smtp-01.tld.t-online.de]"]}
test=pd.DataFrame(data)
test.head()

,os
0,"[smtp-02.tld.t-online.de, smtp-02.tld.t-online..."
1,[smtp-01.tld.t-online.de]
2,"[smtp-02.tld.t-online.de,smtp-01.tld.t-online.de]"


In [5]:
#main_data["length_mx"]=main_data["MX-Record from the name server"].str.len()
test["length_mx"] = test["os"].apply(lambda x: len(x))

In [6]:
test.head()

,os,length_mx
0,"[smtp-02.tld.t-online.de, smtp-02.tld.t-online...",3
1,[smtp-01.tld.t-online.de],25
2,"[smtp-02.tld.t-online.de,smtp-01.tld.t-online.de]",49


In [ ]:
test2=main_data["MX-Record from the name server"].str.split("\"",expand=True)
test2.head(20)

In [14]:
main_data_2["length_mx"]=main_data_2["MX-Record from the name server"].apply(lambda x: len(x))

In [16]:
main_data_2.head(20)

,Top level domain,MX-Record from the name server,A-Record of the specific domain,Timestamp of scraping process,length_mx
0,0--1.de,"[mail.0--1.de, mxf993.netcup.net]","[""46.38.249.145""]",2020-12-13 15:36:05,2
1,0--2.de,"[mxf993.netcup.net, mail.0--2.de]","[""212.227.212.163""]",2020-12-13 15:36:05,2
2,0-0-0-0-0.de,"[smtp-02.tld.t-online.de, smtp-01.tld.t-online...","[""80.150.6.143""]",2020-12-13 15:36:05,2
3,0-0-0-1.de,"[smtp-02.tld.t-online.de, smtp-01.tld.t-online...","[""80.150.6.143""]",2020-12-13 15:36:05,2
4,0-0.de,[],"[""185.53.178.13""]",2020-12-13 15:36:05,0
5,0-01.de,[mail.0-01.de],"[""193.34.145.200""]",2020-12-13 15:36:05,1
6,0-1.de,[localhost],"[""91.195.241.137""]",2020-12-13 15:36:05,1
7,0-10.de,[],[],2020-12-13 15:36:05,0
8,0-1000.de,[localhost],"[""91.195.241.137""]",2020-12-13 15:36:06,1
9,0-100kmh.de,[smtpin.rzone.de],"[""81.169.145.95""]",2020-12-13 15:36:06,1


In [ ]:
main_data["A-Record of the specific domain"].value_counts()

* es gibt eine Kummulation bei bestimmten A-Records. Das bedeutet, dass verschiedenste Domains die gleiche IP Adresse haben(oder gar keine???).
* Teileweise gibt es mehrere A-Records pro Domain

### Zusätzliche Datensätze

In [ ]:
asnip4=pd.read_csv("GeoLite2-ASN-CSV_20210608/GeoLite2-ASN-Blocks-IPv4.csv")
asnip4.head()

In [ ]:
asnip4[asnip4["autonomous_system_organization"].str.match(".*?FLAR")==True]

In [ ]:
asnip6=pd.read_csv("GeoLite2-ASN-CSV_20210608/GeoLite2-ASN-Blocks-IPv6.csv")
asnip6.head()

In [ ]:
cityip4=pd.read_csv("GeoLite2-City-CSV_20210608/GeoLite2-City-Blocks-IPv4.csv")
cityip4.head()

In [ ]:
cityip6=pd.read_csv("GeoLite2-City-CSV_20210608/GeoLite2-City-Blocks-IPv6.csv")
cityip6.head()

In [ ]:
cityloc=pd.read_csv("GeoLite2-City-CSV_20210608/GeoLite2-City-Locations-de.csv")
cityloc.head()

In [ ]:
countryip4=pd.read_csv("GeoLite2-Country-CSV_20210608/GeoLite2-Country-Blocks-IPv4.csv")
countryip4.head()

In [ ]:
countryip6=pd.read_csv("GeoLite2-Country-CSV_20210608/GeoLite2-Country-Blocks-IPv6.csv")
countryip6.head()

In [ ]:
countryloc=pd.read_csv("GeoLite2-Country-CSV_20210608/GeoLite2-Country-Locations-de.csv")
countryloc.head()